# Notes: Circuit Manipulation

### by ReDay Zarra

## Setup

In [1]:
import sys
print(sys.version)

3.9.17 (main, Jun  6 2023, 20:11:21) 
[GCC 11.3.0]


In [2]:
import importlib, pkg_resources
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/lib/python3/dist-packages/pkg_resources/__init__.py'>

In [3]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

2023-07-02 12:56:37.121980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-02 12:56:37.122029: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
2023-07-02 12:56:41.693866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-02 12:56:41.693932: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-02 12:56:41.693976: I tensorflow/stream_executor/cuda/cuda_diagnostics

In [4]:
# Visualization tools: matplotlib and Cirq's svg circuit
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

In [5]:
print("TensorFlow version: ", tf.__version__)
print("TensorFlow Quantum version: ", tfq.__version__)
print("Cirq version: ", cirq.__version__)

TensorFlow version:  2.7.0
TensorFlow Quantum version:  0.7.2
Cirq version:  1.1.0


## Circuit Manipulation

### Patterns for Arguments

Generator: a Python function that uses the yield keyword to return an iterable sequence of items.

In Cirq, `my_layer()` is a generator function that yields a sequence of operations and collections of operations. Each `yield` statement in the function produces a new item in the sequence. These items can be individual operations, lists of operations, or nested lists of operations.

In [8]:
# Define qubits
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]

# Produces a series of quantum operations and collections of operations.
def my_layer():
    yield cirq.CZ(q0, q1)
    yield [cirq.H(q) for q in (q0, q1, q2)]
    yield [cirq.CZ(q1, q2)]
    yield [cirq.H(q0), [cirq.CZ(q1, q2)]]

# Create new circuit and add all operations yielded by the generator
circuit = cirq.Circuit()
circuit.append(my_layer())

for x in my_layer():
    print(x)

CZ(q(0, 0), q(1, 0))
[cirq.H(cirq.GridQubit(0, 0)), cirq.H(cirq.GridQubit(1, 0)), cirq.H(cirq.GridQubit(2, 0))]
[cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H(cirq.GridQubit(0, 0)), [cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]


In [9]:
print(circuit)

(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───


The `Circuit.append` (or `Circuit.insert`) is able to accept an iterator and turn it into a simple, one-dimensional list of operations. This process is called "**flattening**", because it takes a potentially complex, multi-layered structure and **turns it into a simple list**.

`cirq.OP_TREE`: it's a kind of contract or promise about the structure of the data. If you have an iterable structure (like a list, or a generator function) that **can be flattened into a list of operations** (or Moment objects, which are themselves just collections of operations), **then that structure is considered an** `OP_TREE`.

**Sub-circuits**: is just a **part of a larger quantum circuit**, and a generator function is a convenient way to generate these sub-circuits on the fly. Because these generator functions can accept arguments, you can easily adjust the size of the sub-circuits or the parameters of the operations within them, which can make your code much more flexible and powerful.

In [11]:
circuit = cirq.Circuit(cirq.H(q0), cirq.H(q1))
print(circuit)

(0, 0): ───H───

(1, 0): ───H───


### Slicing and Iterating

In Cirq, you can **iterate through every moment within the circuit with a simple** `for` **loop**. Similar to how you can iterate a list of elements. 

In [28]:
# Create a circuit with an OP_TREE inside
circuit = cirq.Circuit(cirq.H(q0), cirq.CZ(q0, q1))

# Print every moment in the circuit
for index, moment in enumerate(circuit):
    print(f"Moment #{index + 1}")
    print(moment)
    print("\n")

Moment #1
  ╷ 0
╶─┼───
0 │ H
  │


Moment #2
  ╷ 0
╶─┼───
0 │ @
  │ │
1 │ @
  │




Circuits in Cirq **can be sliced just like Python lists or arrays**. Each slice operation on a Circuit **returns a new Circuit** that contains only the moments within the specified range.

In [21]:
# Create a ciruit with an OP_TREE inside
circuit = cirq.Circuit(cirq.H(q0), cirq.CZ(q0, q1), cirq.H(q1), cirq.CZ(q0, q1))

# Print a new circuit with only the second and thrid moment inside
print(circuit[1:3])

(0, 0): ───@───────
           │
(1, 0): ───@───H───


### Circuit Operations

`cirq.CircuitOperation`: used to encapsulate a `cirq.Circuit` **as an operation that can be inserted into another** `cirq.Circuit`. This is useful because it allows you to **reuse defined sub-circuits** and significantly simplifies the construction of large, repetitive quantum circuits.

`Circuit.freeze()`: Freezing a circuit **signifies that we're done modifying the sub-circuit** and it is **ready to be used as** a `cirq.CircuitOperation`. Freezing is important especially in terms of serialization, which is the process of converting the circuit's state to a format that can be stored or transmitted and reconstructed later.

In [22]:
# Create a subcircuit containing operations
subcircuit = cirq.Circuit(cirq.H(q1), cirq.CZ(q0, q1), cirq.CZ(q2, q1), cirq.H(q1))

# Freeze the circuit - indicates that we are done modifying it
subcircuit_op = cirq.CircuitOperation(subcircuit.freeze())

# Insert subcircuit_op to main circuit with other operations
circuit = cirq.Circuit(cirq.H(q0), cirq.H(q2), subcircuit_op)
print(circuit)

               [ (0, 0): ───────@─────────── ]
               [                │            ]
(0, 0): ───H───[ (1, 0): ───H───@───@───H─── ]───
               [                    │        ]
               [ (2, 0): ───────────@─────── ]
               │
(1, 0): ───────#2────────────────────────────────
               │
(2, 0): ───H───#3────────────────────────────────


### Frozen Circuit

`cirq.FrozenCircuit`: a type of **circuit that is inherently immutable**, meaning it can't be modified once it's created. This is analogous to using the `freeze()` method on a regular `cirq.Circuit`.

In [23]:
# Create a main circuit then add a frozen circuit (containing operations) inside
circuit = cirq.Circuit(
    cirq.CircuitOperation(
        cirq.FrozenCircuit(cirq.H(q1), cirq.CZ(q0, q1), cirq.CZ(q2, q1), cirq.H(q1))
    )
)

print(circuit)

           [ (0, 0): ───────@─────────── ]
           [                │            ]
(0, 0): ───[ (1, 0): ───H───@───@───H─── ]───
           [                    │        ]
           [ (2, 0): ───────────@─────── ]
           │
(1, 0): ───#2────────────────────────────────
           │
(2, 0): ───#3────────────────────────────────


### Customized Circuit Operations

`cirq.CircuitOperation`: represents **a subcircuit that can be inserted into a larger circuit and customized** in a few ways. This is very useful in quantum circuits where certain sets of gates are repeated multiple times, possibly on different sets of qubits, as you don't need to redefine these sets of operations each time.

In [30]:
# Create a CircuitOperation
subcircuit_op = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.CZ(q0, q1)))

# Create a new CircuitOperation that represents the same circuit repeated twice
repeated_subcircuit_op = subcircuit_op.repeat(2)

# Create a new CircuitOperation that applies operation to different qubits: q0 => q2
moved_subcircuit_op = subcircuit_op.with_qubit_mapping({q0: q2})

# Build a main circuit containing all of the above inside
circuit = cirq.Circuit(subcircuit_op, repeated_subcircuit_op, moved_subcircuit_op)
print(circuit)

           [ (0, 0): ───@─── ]   [ (0, 0): ───@─── ]
(0, 0): ───[            │    ]───[            │    ]────────────────────────────────────────────────────────────────
           [ (1, 0): ───@─── ]   [ (1, 0): ───@─── ](loops=2)
           │                     │
(1, 0): ───#2────────────────────#2─────────────────────────────#2──────────────────────────────────────────────────
                                                                │
                                                                [ (0, 0): ───@─── ]
(2, 0): ────────────────────────────────────────────────────────[            │    ]─────────────────────────────────
                                                                [ (1, 0): ───@─── ](qubit_map={q(0, 0): q(2, 0)})


### Simultaneous Circuit Operations

In Cirq, a `CircuitOperation` is a special type of operation that **encapsulates a** `Circuit` **as its unit of operation**. This means that instead of performing a single gate or operation, a `CircuitOperation` **can execute a whole circuit on its assigned qubits.**

The `CircuitOperation` **can be placed in any** `Moment` **that does not already contain operations on its qubits**, much like a regular operation. Therefore, `CircuitOperation` can be used to depict complex operation timings.

In [32]:
# Create a CircuitOperation 
subcircuit_op = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.H(q0)))

# Build a main circuit containing customized CircuitOperations
circuit = cirq.Circuit(
    subcircuit_op.repeat(3), subcircuit_op.repeat(2).with_qubit_mapping({q0:q1})
)

print(circuit)

(0, 0): ───[ (0, 0): ───H─── ](loops=3)─────────────────────────────────

(1, 0): ───[ (0, 0): ───H─── ](qubit_map={q(0, 0): q(1, 0)}, loops=2)───


Even though the circuits are repeated a different number of times, **they both belong to the same** `Moment` **in the circuit**. This means **they are conceptually executed in parallel, or at the same time**. This is because, from a programming perspective, **the operations are just instructions to be executed**, and we can add them to the same `Moment` regardless of how many times they are repeated.

**However**, when you run this circuit on actual quantum hardware or a simulator, **it may not necessarily hold true that they are executed simultaneously**.

This means that while you can conceptually think of all operations within a `Moment` as happening simultaneously, the actual execution on hardware or a simulator might not strictly adhere to this concept.

### Nested Circuit Operations

Nesting `CircuitOperation`s can be used to **build higher-level structures out of simpler ones**. This allows for the reuse of smaller, modular pieces of a quantum circuit, which can simplify the representation of complex circuits.

In [36]:
# Create CircuitOperations containing each other
qft_1 = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.H(q0)))

qft_2 = cirq.CircuitOperation(cirq.FrozenCircuit(cirq.H(q1), cirq.CZ(q0, q1) ** 0.5, qft_1))

qft_3 = cirq.CircuitOperation(
    cirq.FrozenCircuit(cirq.H(q2), cirq.CZ(q1, q2) ** 0.5, cirq.CZ(q0, q2) ** 0.5, qft_2)
)

print("Original qft_3 CircuitOperation: \n")
print(qft_3)

Original qft_3 CircuitOperation: 

[                                [ (0, 0): ───────@───────[ (0, 0): ───H─── ]─── ]    ]
[ (0, 0): ───────────────@───────[                │                              ]─── ]
[                        │       [ (1, 0): ───H───@^0.5───────────────────────── ]    ]
[                        │       │                                                    ]
[ (1, 0): ───────@───────┼───────#2────────────────────────────────────────────────── ]
[                │       │                                                            ]
[ (2, 0): ───H───@^0.5───@^0.5─────────────────────────────────────────────────────── ]


### Mapped Circuit

`mapped_circuit`: a way of **visualizing the** `CircuitOperation` **as a regular** `Circuit`, with all the mappings and repetitions of its operations applied. This provides a way to **see what the actual quantum circuit would look like**, given the nested CircuitOperation and any modifications like qubit remapping and operation repetition.

In [37]:
# Unroll the outermost layer of qft_3 CircuitOperation to a normal circuit
print("Single layer unroll: \n")
print(qft_3.mapped_circuit(deep = False))

Single layer unroll: 

                               [ (0, 0): ───────@───────[ (0, 0): ───H─── ]─── ]
(0, 0): ───────────────@───────[                │                              ]───
                       │       [ (1, 0): ───H───@^0.5───────────────────────── ]
                       │       │
(1, 0): ───────@───────┼───────#2──────────────────────────────────────────────────
               │       │
(2, 0): ───H───@^0.5───@^0.5───────────────────────────────────────────────────────


This prints the **circuit obtained by unrolling a single layer of** `qft_3`. Any `CircuitOperation`s nested inside `qft_3` are still shown as `CircuitOperation`s, without their individual operations being shown explicitly.

In [38]:
# Unroll all of the CircuitOperations recursively in qft_3
print("Recursive unroll: \n")
print(qft_3.mapped_circuit(deep = True))

Recursive unroll: 

(0, 0): ───────────────@───────────@───────H───
                       │           │
(1, 0): ───────@───────┼───────H───@^0.5───────
               │       │
(2, 0): ───H───@^0.5───@^0.5───────────────────
